In [1]:
# prepare packages
import requests, pandas as pd
from datetime import datetime, timedelta, timezone
from bs4 import BeautifulSoup
import logging
# from pythonjsonlogger import jsonlogger
#import logmatic
import os
from jieba.analyse import extract_tags
import sys
import logstash
import time, random
import pymongo
import jieba



In [2]:
# 建立 MongoClient
host = '10.120.14.105'
client = "mongodb://" + host + ":27017/"
# 多執行續下 可能被 dead lock 
myclient = pymongo.MongoClient(client, connect=False)
mydb = myclient["mydatabase"]
mycol = mydb["Ptt_debug"]

jieba.load_userdict("bigdict.txt")


# insert many to mongo
# x = mycol.insert_many(mylist)
# many should be a list of dict

# 傳回 _id 的值
# print(x.inserted_ids)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.127 seconds.
Prefix dict has been built succesfully.


In [3]:
# memory
#url = "https://www.ptt.cc/bbs/Food/index2.html"

# set proxy to use
proxylist = [{"http": "37.187.120.123:80"},            
             {"https": "178.128.31.153:8080"},
             {"http": "167.114.180.102:8080"},
             {"http": "104.131.214.218:80"},
             {"http": "167.114.196.153:80"},
            {"http":"220.132.207.11:45069"}]
header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}


In [4]:
# 直接傳進 Logstash 在進入 Elasticsearch


logger = logging.getLogger('python-logstash-logger')
logger.setLevel(logging.INFO)
host_demo = "logstash"
logger.addHandler(logstash.TCPLogstashHandler(host_demo,5959))






In [5]:
# 製作 url List
def get_url(n=2,m=3):
    result = []
    # 增加直覺 從第n頁 到 第m頁
    m = m + 1
    for t in range(n,m):
        url = "https://www.ptt.cc/bbs/Food/index" + str(t) + ".html"
        result.append(url)
    return result

In [6]:
# 自定義 404 connectError
class Connect404Except(Exception):
    pass

In [23]:
# slove proxy connect error try next
def get_connect(url):
    i = random.randint(0,5)
    
    while True :
        
        try :
            resp= requests.get(url,headers=header,proxies=proxylist[i])          
            resp.encoding = "utf-8"
            time.sleep(random.randint(1,10)/10)
           
            
        except requests.exceptions.ConnectionError :             
            logger.warning("Proxy Error",exc_info=True,extra={"proxy":proxylist[i],"url":url})
            
            i = (i + 1) % 6
            continue

            
            
        
        if resp.status_code == 200 :
            code = "HTTP response code = " + str(resp.status_code)
            logger.info(code,exc_info=True,extra={"url" : url, "time_use" : resp.elapsed.total_seconds(), "proxy_use":proxylist[i]})
                
            break        
        if resp.status_code == 404 :
            logger.warning("This page is gone nowhere !",exc_info=True,extra={"url":url})
                
            break

    
        
    return resp

In [24]:
def parse(urls):
    # 進入ptt列表 抓取 標題、推文數、網址
    result = []
    
    connect = get_connect(urls)
    try :
        if connect.status_code == 404:
            raise Connect404Except
            
        else :
            soup = BeautifulSoup(connect.text)
            
    except Connect404Except : 
        logger.error("index error",exc_info=True,extra={"index":urls})
        pass
        
    
    if soup :    
        print("進入ptt : ", urls)
        post = soup.find_all("div", class_="r-ent")
        global href,title,author,date_in_content,content
        
        for r in post:
            
            if r.find("div", class_="nrec").text:
                try:
                    push = int(r.find("div", class_="nrec").text)
                except ValueError:
                    pass
            if r.find("a"):
                # 取得網址
                href = "http://www.ptt.cc" + r.find("a")["href"]
                # 取得標題
                title = r.find("a").text

                # 進入第二層 進入每一篇
                # 去除公告 請益 為分類的文章
                if "公告" not in title:
                    if  "請益" not in title:
                        print("現在處理: ", href)
                        
                        article = get_connect(href)
                        try:
                            if article.status_code == 404:
                                raise Connect404Except
                            else:
                                art = BeautifulSoup(article.text)  
                                
                        except Connect404Except:
                            logger.error("Page error",exc_info=True,extra={"url":href,"index":urls})
                            continue
                        
                        time.sleep(random.randint(1,10)/10)
                        
                        
                        
                        if art:
                            content = art.find("div", id="main-content")


                            # 保存要丟棄的資訊
                            val = content.find_all("span", {"class": "article-meta-value"})
                            #  文章排版問題 有些文章沒有部份以下資訊
                            try:
                                author = val[0].text

                                date_in_content = val[3].text
                            except IndexError as e:
                                logger.warning("article span list out of index",exc_info=True, extra={"url": href})
                                author = ""
                                date_in_content = ""

                            # 開始丟棄資訊
                            removes = content.find_all("div", class_="article-metaline")
                            for remove in removes:
                                remove.extract()
                            removes = content.find_all("div", class_="article-metaline-right")
                            for remove in removes:
                                remove.extract()
                            removes = content.find_all("span", class_="f2")
                            for remove in removes:
                                if "※" in remove.text:
                                    remove.extract()
                            # 處理推噓文
                            ps = content.find_all("div", class_="push")
                            score = 0
                            push = 0
                            pos = []
                            neg = []
                            arrow = []
                            for p in ps:
                                tag = p.find("span", class_="push-tag").text
                                if "推" in tag:
                                    score = score + 1
                                    # replace 第三個參數 只置換第一個
                                    # push_content = p.find("span", class_="push-content").text.replace(": ","",1)
                                    push_content = p.find("span", class_="push-content").text
                                    push_ID = p.find("span", class_="push-userid").text
                                    push_total = push_ID + push_content
                                    pos.append(push_total)
                                elif "噓" in tag:
                                    score = score - 1
                                    push_content = p.find("span", class_="push-content").text
                                    push_ID = p.find("span", class_="push-userid").text
                                    push_total = push_ID + push_content
                                    neg.append(push_total)
                                else:
                                    push_content = p.find("span", class_="push-content").text
                                    push_ID = p.find("span", class_="push-userid").text
                                    push_total = push_ID + push_content
                                    arrow.append(push_total)
                                p.extract()


                            try:
                                content = content.text.replace("\r", "").replace("\n", "").replace(" ","")
                            except AttributeError as e:
                                logger.warning("content text cant regularize",exc_info=True, extra={"url": href})
                                
                                                                         
                        
                        
                            result.append({
                                "title": title,
                                "author": author,
                                "date": date_in_content,
                                "push": push,
                                "href": href,
                                "content": content,
                                "score": score,
                                "推文": pos,
                                "噓文": neg,
                                "箭頭": arrow,
                                "tags" : extract_tags(content , 10)

                            })
        try:
            x = mycol.insert_many(result)
            if x.inserted_ids :
                logger.info("Insert Successful",exc_info=True,extra={"count":len(x.inserted_ids),"index_url":urls})
        except Exception as e:
            logger.warning("DB error",exc_info=True,extra={"error_name":e, "index_url" : urls})



    return result

In [ ]:
logger.info("test",exc_info=True)

In [26]:
from concurrent.futures import ThreadPoolExecutor, wait, ProcessPoolExecutor, as_completed

new_result = []
with ThreadPoolExecutor(max_workers= 15,thread_name_prefix="Ptt_crawler") as executor:
    future_result = {executor.submit(parse, url) : url for url in get_url(2,17)}

    

    for future in as_completed(future_result):
        
        logger.info("Worker's job done",exc_info=True)
        
    
    

進入ptt :  https://www.ptt.cc/bbs/Food/index3.html
進入ptt :  https://www.ptt.cc/bbs/Food/index8.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1089756185.A.DF1.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1076520608.A.319.html
進入ptt :  https://www.ptt.cc/bbs/Food/index9.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1091423007.A.8DC.html
進入ptt :  https://www.ptt.cc/bbs/Food/index6.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1083830551.A.BA5.html
進入ptt : 進入ptt :  https://www.ptt.cc/bbs/Food/index16.html
 現在處理:  http://www.ptt.cc/bbs/Food/M.1097422169.A.504.html
進入ptt :  https://www.ptt.cc/bbs/Food/index12.html
進入ptt :  https://www.ptt.cc/bbs/Food/index14.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1096039368.A.B55.html
進入ptt :  https://www.ptt.cc/bbs/Food/index4.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1094518248.A.183.html
進入ptt :  https://www.ptt.cc/bbs/Food/index10.html
https://www.ptt.cc/bbs/Food/index7.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1086428166.A.34F.html
進入ptt :  https://www.ptt.cc/bbs/Food/index

現在處理:  http://www.ptt.cc/bbs/Food/M.1089148033.A.93D.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1084269729.A.76A.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1096303453.A.A0B.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1094397391.A.CF7.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1093084582.A.8EB.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1095851236.A.D13.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1089206891.A.938.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1076297742.A.630.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1091808670.A.721.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1098011238.A.57E.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1076854528.A.3A4.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1096338774.A.279.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1097561823.A.B28.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1094401726.A.309.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1076391807.A.B79.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1095002949.A.3DE.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1077021218.A.B02.html
現在處理:  http://

KeyboardInterrupt: 

In [ ]:
import pandas as pd
df = pd.DataFrame(columns=["title","author","date","push","href","content","score","推文","噓文","箭頭"])
for r in new_result:
    for evey in r:
        s = pd.Series([evey["title"],evey["author"],evey["date"],evey["push"],evey["href"],evey["content"],evey["score"],evey["推文"],evey["噓文"],evey["箭頭"]]
                      ,index=["title","author","date","push","href","content","score","推文","噓文","箭頭"])
        df = df.append(s,ignore_index=True)
        

In [ ]:
df



In [ ]:
import json

collect = json.loads(df.T.to_json()).values()

In [ ]:
from pymongo import MongoClient

class MongoBase:
    def __init__(self,collection):
        self.collection=collection
        self.OpenDB()
    def OpenDB(self):
        user='root'
        passwd='root'
        # on local host
        host='192.168.114.130'
        port='27017'
        auth_db='admin'
        uri = "mongodb://"+user+":"+passwd+"@"+host+":"+port+"/"+auth_db+"?authMechanism=SCRAM-SHA-1"
        self.con = MongoClient(uri, connect=True)
        self.db=self.con['wangdong']
        self.collection=self.db[self.collection]
    def closeDB(self):
        self.con.close()
        
mongo = MongoBase('test3')

In [ ]:


mongo.collection.insert_many(collect)
mongo.closeDB()


In [ ]:
parse("https://www.ptt.cc/bbs/Food/index7004.html")

In [ ]:
# 進入ptt列表 抓取 標題、推文數、網址 
result = []
soup = BeautifulSoup(get_connect(url))
print("進入ptt : ",url)
post = soup.find_all("div", class_="r-ent")
for r in post:
            # 取得推的數量
    push = 0
    if r.find("div", class_="nrec").text:
        try:
            push = int(r.find("div", class_="nrec").text)
        except ValueError:
                pass
    if r.find("a"):
        # 取得網址
        href = "http://www.ptt.cc" + r.find("a")["href"]
        # 取得標題
        title = r.find("a").text
        
        # 進入第二層 進入每一篇
        # 去除公告 請益 為分類的文章
        if not "公告" in title :
            if not "請益" in title:
                print("現在處理: ", href)
                art = BeautifulSoup(get_connect(href))
                article = []
                content = art.find("div", id="main-content")
                # 保存要丟棄的資訊
                val = content.find_all("span", {"class":"article-meta-value"})
                author = val[0].text
                title_in_content = val[2].text
                date_in_content = val[3].text
                
                # 開始丟棄資訊
                removes = content.find_all("div", class_="article-metaline")
                for remove in removes:
                    remove.extract()
                removes = content.find_all("div", class_="article-metaline-right")
                for remove in removes:
                    remove.extract()
                removes = content.find_all("span", class_="f2")
                for remove in removes:
                    remove.extract()
                removes = content.find_all("div", class_="push")
                for remove in removes:
                    remove.extract()
                article.append({
                    "author" : author,
                    "title_in_content" : title_in_content,
                    "date_in_content" : date_in_content,
                    "content" : content.text
                })
                
    result.append({
        "title" : title,
        "push" : push,
        "href" : href,
        "content" : article
        
    })
    
    time.sleep(random.randint(1,6))
                






In [ ]:
import json
# 轉成 Dataframe
df = pd.DataFrame(result)



In [ ]:
# 排列欄位順序

df = df[ ["title", "push", "href", "content" ] ]

In [ ]:
df.iloc[1]

In [ ]:
# I/O 較多 使用 thread
start = time.time()
with ThreadPoolExecutor(max_workers=4) as executor:
    results = executor.map(int, ['1', '2', '3', '4', '5'])
    for v in results:
        print(v)
end = time.time()
print("time used :",end - start)